<a href="https://colab.research.google.com/github/AlexChek51/Neuro_Consultant_for_Construction/blob/main/Neuro_Consultant_for_Construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Сначала необходимо установить все необходимые библиотеки и фреймворки.

In [ ]:
!pip install openai gradio tiktoken langchain langchain-openai langchain-community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 83.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.7 MB/

Устанавливаем ключ для OpenAI.

In [ ]:
import getpass # для работы с паролями
import os      # для работы с окружением и файловой системой
from langchain_openai import ChatOpenAI

# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")
model = ChatOpenAI(model="gpt-3.5-turbo")

Введите OpenAI API Key:··········


Перед созданием обучающего файла для нейро-консультанта сделаем запрос в ChatGPT для выявления структуры данных.

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

result = model.invoke([HumanMessage(content="Необходимо структурировать документ, для обучения нейро-консультанта по строительству домов")])
parser.invoke(result)

'Документация для обучения нейро-консультанта по строительству домов:\n\nI. Введение\n- Цель и задачи обучения\n- Описание роли нейро-консультанта в строительстве домов\n- Важность использования нейротехнологий в данной области\n\nII. Основы строительства домов\n- Основные этапы строительства дома\n- Требования к материалам и технологиям\n- Знание строительных стандартов и нормативов\n\nIII. Нейротехнологии в строительстве\n- Как применять нейросети для оптимизации процесса строительства\n- Использование искусственного интеллекта для улучшения производительности\n- Роль виртуальной и дополненной реальности в строительстве\n\nIV. Консультирование клиентов\n- Взаимодействие с заказчиками и анализ их потребностей\n- Разработка индивидуальных проектов с учетом нейротехнологий\n- Поддержка клиентов на всех этапах строительства\n\nV. Практические навыки\n- Обучение работе с программным обеспечением для моделирования и проектирования домов\n- Практические задания по оптимизации процесса строи

В данном задании нет необходимости использовать Class и список поддерживаемых нейро-сотрудников т.к. разрабатываем одного нейро-консультанта. Удалил подсчет числа токенов (можно обойтись без этого, на роботоспособность код не влияет).

In [ ]:
# Отправка запросов
import requests
# Разделение текста на куски или чанки (chunk)
from langchain.text_splitter import CharacterTextSplitter
# Работа с документами в langchain
from langchain.docstore.document import Document
# Эмбеддинги для OpenAI
from langchain_openai import OpenAIEmbeddings
# Доступ к векторной базе данных
from langchain.vectorstores import Chroma

# Глобальная переменная для хранения базы данных эмбеддингов
search_index = None

# Метод загрузки текстового документа в векторную базу знаний
def load_search_indexes(url):
    # Скачиваем гугл документ в текстовом формате
    response = requests.get(url)

    # Извлекаем данные как текст
    text = response.text

    # Вызываем метод векторизации текста и сохранения в векторную базу данных
    return create_embedding(text)

# Метод разбора текста и его сохранение в векторную базу знаний
def create_embedding(text):
    source_chunks = []
    splitter = CharacterTextSplitter(separator="\n", chunk_size=1024, chunk_overlap=0)

    # Применяем splitter (функцию расщепления) к данным и перебираем все получившиеся чанки (фрагменты)
    for chunk in splitter.split_text(text):
        # LangChain работает с документами, поэтому из текстовых чанков мы создаем фрагменты документов
        source_chunks.append(Document(page_content=chunk, metadata={}))

    search_index = Chroma.from_documents(source_chunks, OpenAIEmbeddings())

    return search_index

# Функция для обучения модели
def train_model():
    global search_index
    doc_url = "https://docs.google.com/document/d/1UHRW65WUjzlyQEq9xanpOEgtGYK5Ui7f/export?format=txt"
    search_index = load_search_indexes(doc_url)
    return "Модель обучена!"

# Функция для обработки пользовательского запроса
def handle_query(user_query):
    global search_index
    if search_index is None:
        return "Модель не обучена. Пожалуйста, обучите модель перед выполнением запросов."

    # Ищем релевантные документы
    relevant_docs = search_index.similarity_search(user_query)

    # Составляем ответ на основе найденных документов
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    response = model.invoke([HumanMessage(content=f"Контекст: {context}\n\nВопрос: {user_query}")])

    return parser.invoke(response)

Выбор сотрудника сделал не изменяемым. Удалил Label выбранное имя. Убрал логирование, т.к. обычному пользователю, это не обязательно видеть. Это так же позволило уменьшить и оптимизировать код. Добавил статус обучения модели.

In [ ]:
# Отприсовка интерфейса с помощью grad
import gradio as gr

prompt_messange = "Ты консультант по строительству домов, к тебе могут обращаться люди разных возростов и профессий за подсказками и ответами на их вопросы в чате компании. Постарайся дать развернутый ответ, твоя задача ответить так, чтобы у человека не осталось больше вопросов к тебе. Отвечай по существу, без лишних эмоций и слов, от тебя нужна только точная информация. Отвечай максимально точно по документу, не придумывай ничего от себя."

# Gradio позволяет объединять элементы в блоки
blocks = gr.Blocks()

# Работаем с блоком
with blocks as demo:
    # Объявляем элемент
    subject = gr.Textbox(label="Сотрудник", value='Нейро-консультант по строительству домов', interactive=False)
    # Промпт для запроса к MLL
    prompt = gr.Textbox(label="Промт", value=prompt_messange, interactive=True)
    # Ссылка на файл обучения
    link = gr.HTML(value='<a href="https://docs.google.com/document/d/1UHRW65WUjzlyQEq9xanpOEgtGYK5Ui7f/edit" target="_blank">Документ с информацией для ответа клиенту</a>')
    # Поле пользовательского запроса к MLL
    query = gr.Textbox(label="Запрос к LLM (замените на свой)", value='Какой материал, выбрать для строительства крыши?', interactive=True)

    # Строку в gradio можно разделить на столбцы (каждая кнопка в своем столбце)
    with gr.Row():
        train_btn = gr.Button("Обучить модель")       # кнопка запуска обучения
        request_btn = gr.Button("Запрос к модели")    # кнопка отправки запроса к LLM

    with gr.Row():
        train_status = gr.Textbox(label="Статус обучения", interactive=False)
        response = gr.Textbox(label="Ответ LLM") # Текстовое поле с ответом от LLM

    # Привязка функции train_model к кнопке train_btn
    train_btn.click(train_model, inputs=[], outputs=train_status)

    # Привязка функции handle_query к кнопке request_btn
    request_btn.click(handle_query, inputs=[query], outputs=[response])

# Запуск приложения
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://a0e8107eaf0605e754.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
